# TAREA 4: ETL para Proveedores y Movimientos

##  0. Configuración Inicial

### 0.1 Scripts de la DB 
Este script "reinicia" la db con las tablas, y borra registros si a lugar para ser la primera vez que se corre el ETL. </br>
[DB Script](https://github.com/EduardoToledoMIAD/ETL/blob/main/Semana_4/Configuracion%20Schema.sql)

A continuacion , se observa el estado de la bd , una vez fue inicializada con el anterior script:

![](https://raw.githubusercontent.com/EduardoToledoMIAD/ETL/main/Semana_4/images/DB%20Status.png)

### 0.2 Configuracion JDBC

In [1]:
# Configuración servidor base de datos transaccional
db_user = 'Estudiante_111_202315'
db_psswd = 'aabb1122'
source_db_connection_string = 'jdbc:mysql://157.253.236.116:8080/WWImportersTransactional'
dest_db_connection_string = 'jdbc:mysql://157.253.236.116:8080/Estudiante_111_202315'



db_connection_string = 'jdbc:mysql://157.253.236.116:8080/WWImportersTransactional'
# Driver de conexion
path_jar_driver = 'C:\Program Files (x86)\MySQL\Connector J 8.0\mysql-connector-java-8.0.28.jar'

PATH='./'

In [2]:
import os 
from pyspark.sql import functions as f, SparkSession, types as t
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql.functions import udf, col, length, isnan, when, count, regexp_replace
from datetime import datetime
from pyspark.sql.functions import lit
from pyspark.sql.types import TimestampType, DateType
import re
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, TimestampType,IntegerType,StringType,DecimalType
from pyspark.sql.functions import col, date_format, to_date



### 0.2 Creacion session de Spark

In [3]:
#Configuración de la sesión
conf=SparkConf() \
    .set('spark.driver.extraClassPath', path_jar_driver)

spark_context = SparkContext(conf=conf)
sql_context = SQLContext(spark_context)
spark = sql_context.sparkSession

C:\Users\estudiante\anaconda3\envs\Tutoriales\lib\site-packages\pyspark\sql\context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


### 0.3 Conexión y carga de datos

Se define la función para conexión y cargue de dataframes desde la base de datos origen y luego la función para guardar un dataframe en una tabla de la base de datos destino.

In [4]:
def obterner_dataframe_desde_csv(_PATH, _sep):
    return spark.read.load(_PATH, format="csv", sep=_sep, inferSchema="true", header='true')

def obtener_dataframe_de_bd(db_connection_string, sql, db_user, db_psswd):
    df_bd = spark.read.format('jdbc')\
        .option('url', db_connection_string) \
        .option('dbtable', sql) \
        .option('user', db_user) \
        .option('password', db_psswd) \
        .option('driver', 'com.mysql.cj.jdbc.Driver') \
        .load()
    return df_bd

def guardar_db(db_connection_string, df, tabla, db_user, db_psswd):
    df.select('*').write.format('jdbc') \
      .mode('append') \
      .option('url', db_connection_string) \
      .option('dbtable', tabla) \
      .option('user', db_user) \
      .option('password', db_psswd) \
      .option('driver', 'com.mysql.cj.jdbc.Driver') \
      .save()

### 1. Tabla Dimensional: TIPOTRANSACTION

Fuente de datos:
Tabla transactional TiposTransaction . Su fuente de datos es la tabla transaccional <i>TiposTransaccion</i>


#### 1.1 Extracción


In [5]:
sql_tipostrans = '''(SELECT DISTINCT TipoTransaccionID , TipoTransaccionNombre  FROM WWImportersTransactional.TiposTransaccion) AS Temp_empleados'''
tipos_trans_df = obtener_dataframe_de_bd(source_db_connection_string, sql_tipostrans, db_user, db_psswd)
tipos_trans_df.show(10)
tipos_trans_df.printSchema()

print(f'Numero de registros {tipos_trans_df.count()}');

+-----------------+---------------------+
|TipoTransaccionID|TipoTransaccionNombre|
+-----------------+---------------------+
|                2| Customer Credit Note|
|                3| Customer Payment ...|
|                4|      Customer Refund|
|                5|     Supplier Invoice|
|                6| Supplier Credit Note|
|                7| Supplier Payment ...|
|                8|      Supplier Refund|
|                9|       Stock Transfer|
|               10|          Stock Issue|
|               11|        Stock Receipt|
+-----------------+---------------------+
only showing top 10 rows

root
 |-- TipoTransaccionID: integer (nullable = true)
 |-- TipoTransaccionNombre: string (nullable = true)

Numero de registros 12


#### 1.2 Transformación
Se cambian los nombres de las columnas fuente  a los nombres adecuados en la bodega: ID_Tipo_transaccion_T, Tipo

In [6]:
# TRANSFORMACION
tipos_trans_df= tipos_trans_df.selectExpr("TipoTransaccionID AS ID_Tipo_transaccion_T", "TipoTransaccionNombre AS Tipo")
tipos_trans_df.printSchema()



root
 |-- ID_Tipo_transaccion_T: integer (nullable = true)
 |-- Tipo: string (nullable = true)



#### 1.3 Carga


In [7]:
# CARGUE
guardar_db(dest_db_connection_string, tipos_trans_df,'Estudiante_111_202315.TipoTransaccion', db_user, db_psswd)


#### 1.4 Validacion

In [8]:
test = obtener_dataframe_de_bd(dest_db_connection_string, "Estudiante_111_202315.TipoTransaccion", db_user, db_psswd)
test.show(5)
print(f'Numero de registros {test.count()}');

+-----------------------+---------------------+--------------------+
|ID_Tipo_transaccion_DWH|ID_Tipo_transaccion_T|                Tipo|
+-----------------------+---------------------+--------------------+
|                      1|                    2|Customer Credit Note|
|                      2|                    3|Customer Payment ...|
|                      3|                    4|     Customer Refund|
|                      4|                    5|    Supplier Invoice|
|                      5|                    6|Supplier Credit Note|
+-----------------------+---------------------+--------------------+
only showing top 5 rows

Numero de registros 12


#### 1.5 Evidencia MySQL

Fueron almacenados 12 registros desde este notebook.


![Numero Registros  TipoTransaction MySql](https://raw.githubusercontent.com/EduardoToledoMIAD/ETL/main/Semana_4/images/TipoTransaccion_MySQL.png)

### 2. Dimension: CIUDAD
Su fuente de datos es una combinación de las tablas transaccionales <i>paises, provinciasEstados y ciudades</i>

#### 2.1 Extracción

In [9]:
sql_paises = '''(SELECT DISTINCT ID_Pais, Nombre, Continente, Region, Subregion FROM WWImportersTransactional.Paises) AS Temp_paises'''
sql_provincias_estados = '''(SELECT DISTINCT ID_EstadosProvincias AS ID_EstadoProvincia, NombreEstadoProvincia, TerritorioVentas, ID_Pais FROM WWImportersTransactional.EstadosProvincias) AS Temp_estados_provincias'''
sql_ciudades = '''(SELECT DISTINCT ID_ciudad as ID_Ciudad_T, NombreCiudad, ID_EstadoProvincia, Poblacion FROM WWImportersTransactional.Ciudades) AS Temp_ciudades'''

paises_df = obtener_dataframe_de_bd(source_db_connection_string, sql_paises, db_user, db_psswd)
provincias_estados_df = obtener_dataframe_de_bd(source_db_connection_string, sql_provincias_estados, db_user, db_psswd)
ciudades_df = obtener_dataframe_de_bd(source_db_connection_string, sql_ciudades, db_user, db_psswd)

print(ciudades_df.columns, paises_df.columns, provincias_estados_df.columns)

['ID_Ciudad_T', 'NombreCiudad', 'ID_EstadoProvincia', 'Poblacion'] ['ID_Pais', 'Nombre', 'Continente', 'Region', 'Subregion'] ['ID_EstadoProvincia', 'NombreEstadoProvincia', 'TerritorioVentas', 'ID_Pais']


#### 2.2. Transformación
- Cruzar con EstadoProvincias y Paises
- Asignar un ID unico usando la funcion: monotonically_increasing_id

In [10]:
# TRANSFORMACION
ciudades_df = ciudades_df.join(provincias_estados_df, how = 'left', on = 'ID_EstadoProvincia')
ciudades_df = ciudades_df.join(paises_df, how = 'left', on = 'ID_Pais')
ciudades_df = ciudades_df.coalesce(1).withColumn('ID_Ciudad_DWH', f.monotonically_increasing_id() + 1)
ciudades_df = ciudades_df.select('ID_Ciudad_DWH','ID_ciudad_T','NombreCiudad','Continente','Nombre','Poblacion',
                          'Region','TerritorioVentas','NombreEstadoProvincia','Subregion') \
                    .withColumnRenamed('Nombre','Pais')

ciudades_df.printSchema()

ciudades_df.head(3)

root
 |-- ID_Ciudad_DWH: long (nullable = false)
 |-- ID_ciudad_T: integer (nullable = true)
 |-- NombreCiudad: string (nullable = true)
 |-- Continente: string (nullable = true)
 |-- Pais: string (nullable = true)
 |-- Poblacion: long (nullable = true)
 |-- Region: string (nullable = true)
 |-- TerritorioVentas: string (nullable = true)
 |-- NombreEstadoProvincia: string (nullable = true)
 |-- Subregion: string (nullable = true)



[Row(ID_Ciudad_DWH=1, ID_ciudad_T=49, NombreCiudad='Absecon', Continente='North America', Pais='United States', Poblacion=8411, Region='Americas', TerritorioVentas='Mideast', NombreEstadoProvincia='New Jersey', Subregion='Northern America'),
 Row(ID_Ciudad_DWH=2, ID_ciudad_T=150, NombreCiudad='Adelphia', Continente='North America', Pais='United States', Poblacion=None, Region='Americas', TerritorioVentas='Mideast', NombreEstadoProvincia='New Jersey', Subregion='Northern America'),
 Row(ID_Ciudad_DWH=3, ID_ciudad_T=336, NombreCiudad='Albion', Continente='North America', Pais='United States', Poblacion=None, Region='Americas', TerritorioVentas='Mideast', NombreEstadoProvincia='New Jersey', Subregion='Northern America')]

#### 2.3 Carga

In [11]:
# CARGA
guardar_db(dest_db_connection_string, ciudades_df,'Estudiante_111_202315.Ciudad', db_user, db_psswd)

#### 2.4 Validacion

In [12]:
#guardar_db(dest_db_connection_string, proveedores_df,'Estudiante_111_202315.Proveedor', db_user, db_psswd)
test = obtener_dataframe_de_bd(dest_db_connection_string, "Estudiante_111_202315.Ciudad", db_user, db_psswd)
test.show(5)
print(f'Numero de registros {test.count()}');

+-------------+-----------+------------+-------------+-------------+---------+--------+----------------+---------------------+----------------+
|ID_Ciudad_DWH|ID_Ciudad_T|NombreCiudad|   Continente|         Pais|Poblacion|  Region|TerritorioVentas|NombreEstadoProvincia|       Subregion|
+-------------+-----------+------------+-------------+-------------+---------+--------+----------------+---------------------+----------------+
|            1|         49|     Absecon|North America|United States|     8411|Americas|         Mideast|           New Jersey|Northern America|
|            2|        150|    Adelphia|North America|United States|     null|Americas|         Mideast|           New Jersey|Northern America|
|            3|        336|      Albion|North America|United States|     null|Americas|         Mideast|           New Jersey|Northern America|
|            4|        458|   Allamuchy|North America|United States|       78|Americas|         Mideast|           New Jersey|Northern A

#### 2.5 Evidencia MySQL

Fueron almacenados 37940 registros desde este notebook.


![Numero Registros  Ciudad MySql](https://raw.githubusercontent.com/EduardoToledoMIAD/ETL/main/Semana_4/images/Ciudad_MySQL.jpg)

### 3. Dimension: PROVEEDOR
Su fuente de datos es una combinación de las tablas transaccionales <i>proveedoresCopia, CategoriasProveedores y Personas</i>

#### 3.1 Extracción

In [13]:
#EXTRACCION
sql_provedores=  '''(SELECT DISTINCT ProveedorID as ID_Proveedor_T, NombreProveedor AS Nombre,CategoriaProveedorID, PersonaContactoPrincipalID, DiasPago AS Dias_pago, CodigoPostal AS Codigo_postal FROM WWImportersTransactional.proveedores) AS Temp_proveedores'''
sql_cat_prov=  '''(SELECT DISTINCT CategoriaProveedorID, CategoriaProveedor AS Categoria FROM WWImportersTransactional.CategoriasProveedores) AS Temp_cat_proveedores'''
sql_personas=  '''(SELECT DISTINCT ID_persona, NombreCompleto AS Contacto_principal FROM WWImportersTransactional.Personas) AS Temp_personas'''
proveedores_df = obtener_dataframe_de_bd(source_db_connection_string, sql_provedores, db_user, db_psswd)
cat_proveedores_df = obtener_dataframe_de_bd(source_db_connection_string, sql_cat_prov, db_user, db_psswd)
personas_df = obtener_dataframe_de_bd(source_db_connection_string, sql_personas, db_user, db_psswd)
proveedores_df.printSchema()
cat_proveedores_df.printSchema()
personas_df.printSchema()
proveedores_df.count()
proveedores_df.show(5)

root
 |-- ID_Proveedor_T: integer (nullable = true)
 |-- Nombre: string (nullable = true)
 |-- CategoriaProveedorID: integer (nullable = true)
 |-- PersonaContactoPrincipalID: integer (nullable = true)
 |-- Dias_pago: integer (nullable = true)
 |-- Codigo_postal: integer (nullable = true)

root
 |-- CategoriaProveedorID: integer (nullable = true)
 |-- Categoria: string (nullable = true)

root
 |-- ID_persona: integer (nullable = true)
 |-- Contacto_principal: string (nullable = true)

+--------------+--------------------+--------------------+--------------------------+---------+-------------+
|ID_Proveedor_T|              Nombre|CategoriaProveedorID|PersonaContactoPrincipalID|Dias_pago|Codigo_postal|
+--------------+--------------------+--------------------+--------------------------+---------+-------------+
|             4|      Fabrikam, Inc.|                   4|                        27|       30|        40351|
|             5|Graphic Design In...|                   2|            

#### 3.2 Transformación



##### 3.2.1 El código postal igual para todos nuestros proveedores es un error que también fue corregido. [VALIDACION]

In [14]:
# El código postal igual para todos nuestros proveedores es un error que también fue corregido.
grupo_df = proveedores_df.groupBy("Codigo_postal")
cuentagrupos_df = grupo_df.count()
#Obtener si hay duplicados
cuentagrupos_df.filter(cuentagrupos_df["count"] > 1).show()
# Todos los codigos postales son difrentes excepto uno que se repite


+-------------+-----+
|Codigo_postal|count|
+-------------+-----+
|        94101|    2|
+-------------+-----+



##### 3.2.1 Transformacion T1
Cruzar con tablas transaccionales de  CategoriaProveedores y Personas

In [15]:
# TRANSFORMACION: T1

proveedores_df= proveedores_df.join(cat_proveedores_df, how='left', on ='CategoriaProveedorID')
proveedores_df= proveedores_df.join(personas_df, how='left', on = proveedores_df.PersonaContactoPrincipalID == personas_df.ID_persona)
proveedores_df= proveedores_df.select('ID_Proveedor_T','Nombre', 'Categoria','Contacto_principal','Dias_pago','Codigo_postal')
proveedores_df.printSchema()

proveedores_df.head(3)

print(proveedores_df.count())

root
 |-- ID_Proveedor_T: integer (nullable = true)
 |-- Nombre: string (nullable = true)
 |-- Categoria: string (nullable = true)
 |-- Contacto_principal: string (nullable = true)
 |-- Dias_pago: integer (nullable = true)
 |-- Codigo_postal: integer (nullable = true)

13


##### 3.2.2 Transformacion T2
Dias_pago no puedene ser negativos. Multiplicar por -1

In [16]:
proveedores_df = proveedores_df.withColumn("Dias_pago", when(col("Dias_pago") < 0, expr("Dias_pago * -1")).otherwise(col("Dias_pago")))
proveedores_df.filter(col('Dias_pago') >= 0).show()

+--------------+--------------------+--------------------+------------------+---------+-------------+
|ID_Proveedor_T|              Nombre|           Categoria|Contacto_principal|Dias_pago|Codigo_postal|
+--------------+--------------------+--------------------+------------------+---------+-------------+
|             6| Humongous Insurance|servicios de seguros|Madelaine  Cartier|       14|        37770|
|             4|      Fabrikam, Inc.|                ropa|       Bill Lawson|       30|        40351|
|            11|       Trey Research|servicios de mark...|      Donald Jones|        7|        57543|
|            12|   The Phone Company| productos novedosos|           Hai Dam|       30|        56732|
|             9|      Nod Publishers| productos novedosos|      Marcos Costa|        7|        27906|
|             8|  Lucerne Publishing| productos novedosos|       Prem Prabhu|       30|        37659|
|             2|       Contoso, Ltd.| productos novedosos|   Hanna Mihhailov|     

##### 3.2.3 Transformacion T3
En Movimientos, hay filas con proveedorId vacio. Este vacio sera representado con valor 0 en la dimension Proveedorees. . Crear el registro para proveedor vacio.

In [17]:
# Crea el registro para proveedor = nulo

schema = StructType([
    StructField("ID_Proveedor_T", IntegerType(), nullable=True),
    StructField("Nombre", StringType(), nullable=True),
    StructField("Categoria", StringType(), nullable=True),
    StructField("Contacto_principal", StringType(), nullable=True),
    StructField("Dias_pago", IntegerType(), nullable=True),
    StructField("Codigo_postal", IntegerType(), nullable=True)
    
])
data = [(0, 'Missing', 'Missing','Missing',None,None)]
proveedor_nulo_df = spark.createDataFrame(data, schema)
proveedores_df = proveedor_nulo_df.union(proveedores_df)
print(proveedores_df.head(3))
proveedores_df.count()

[Row(ID_Proveedor_T=0, Nombre='Missing', Categoria='Missing', Contacto_principal='Missing', Dias_pago=None, Codigo_postal=None), Row(ID_Proveedor_T=4, Nombre='Fabrikam, Inc.', Categoria='ropa', Contacto_principal='Bill Lawson', Dias_pago=30, Codigo_postal=40351), Row(ID_Proveedor_T=5, Nombre='Graphic Design Institute', Categoria='productos novedosos', Contacto_principal='Penny Buck', Dias_pago=14, Codigo_postal=64847)]


14

#### 3.3. Carga

In [19]:
# CARGUE
guardar_db(dest_db_connection_string, proveedores_df,'Estudiante_111_202315.Proveedor', db_user, db_psswd)

#### 3.4 Validacion

In [20]:
#guardar_db(dest_db_connection_string, proveedores_df,'Estudiante_111_202315.Proveedor', db_user, db_psswd)
test = obtener_dataframe_de_bd(dest_db_connection_string, "Estudiante_111_202315.Proveedor", db_user, db_psswd)
test.show(3)
print(f'Numero de registros {test.count()}');

+----------------+--------------+-------------------+--------------------+------------------+---------+-------------+
|ID_Proveedor_DWH|ID_Proveedor_T|             Nombre|           Categoria|Contacto_principal|Dias_pago|Codigo_postal|
+----------------+--------------+-------------------+--------------------+------------------+---------+-------------+
|               1|             6|Humongous Insurance|servicios de seguros|Madelaine  Cartier|       14|        37770|
|               2|             4|     Fabrikam, Inc.|                ropa|       Bill Lawson|       30|        40351|
|               3|            11|      Trey Research|servicios de mark...|      Donald Jones|        7|        57543|
+----------------+--------------+-------------------+--------------------+------------------+---------+-------------+
only showing top 3 rows

Numero de registros 14


#### 3.5 Evidencia MySQL

Fueron almacenados 14 registros desde este notebook.


![Numero Registros  Proveedor MySql](https://raw.githubusercontent.com/EduardoToledoMIAD/ETL/main/Semana_4/images/Proveedor_MySQL.png)

### 4. Dimension :CLIENTE
Su fuente de datos es una combinación de las tablas transaccionales <i>CategoriasCliente, GruposCompra y Clientes y la tabla dimensional de Ciudad</i>

#### 4.1 Extracción

In [21]:
#EXTRACCION

sql_categoriasCliente = '''(SELECT DISTINCT ID_Categoria, NombreCategoria FROM WWImportersTransactional.CategoriasCliente) AS Temp_categoriasclientes'''
sql_gruposCompra = '''(SELECT DISTINCT ID_GrupoCompra, NombreGrupoCompra FROM WWImportersTransactional.GruposCompra) AS Temp_gruposcompra'''
sql_ciudades =  '''(SELECT DISTINCT * FROM Estudiante_111_202315.Ciudad) AS Temp_ciudad'''
sql_clientes = '''(SELECT DISTINCT ID_Cliente as ID_Cliente_T, Nombre, ClienteFactura, ID_Categoria, ID_GrupoCompra, ID_CiudadEntrega AS ID_Ciudad_T, LimiteCredito, FechaAperturaCuenta, DiasPago FROM WWImportersTransactional.Clientes) AS Temp_clientes'''

categoriasCliente_df = obtener_dataframe_de_bd(source_db_connection_string, sql_categoriasCliente, db_user, db_psswd)
gruposCompra_df = obtener_dataframe_de_bd(source_db_connection_string, sql_gruposCompra, db_user, db_psswd)
ciudades_dwh_df= obtener_dataframe_de_bd(dest_db_connection_string, sql_ciudades, db_user, db_psswd)
clientes_df = obtener_dataframe_de_bd(source_db_connection_string, sql_clientes, db_user, db_psswd)
categoriasCliente_df.printSchema()
gruposCompra_df.printSchema()
ciudades_dwh_df.printSchema()
clientes_df.printSchema()

root
 |-- ID_Categoria: integer (nullable = true)
 |-- NombreCategoria: string (nullable = true)

root
 |-- ID_GrupoCompra: integer (nullable = true)
 |-- NombreGrupoCompra: string (nullable = true)

root
 |-- ID_Ciudad_DWH: long (nullable = true)
 |-- ID_Ciudad_T: integer (nullable = true)
 |-- NombreCiudad: string (nullable = true)
 |-- Continente: string (nullable = true)
 |-- Pais: string (nullable = true)
 |-- Poblacion: long (nullable = true)
 |-- Region: string (nullable = true)
 |-- TerritorioVentas: string (nullable = true)
 |-- NombreEstadoProvincia: string (nullable = true)
 |-- Subregion: string (nullable = true)

root
 |-- ID_Cliente_T: integer (nullable = true)
 |-- Nombre: string (nullable = true)
 |-- ClienteFactura: integer (nullable = true)
 |-- ID_Categoria: integer (nullable = true)
 |-- ID_GrupoCompra: integer (nullable = true)
 |-- ID_Ciudad_T: integer (nullable = true)
 |-- LimiteCredito: decimal(10,0) (nullable = true)
 |-- FechaAperturaCuenta: timestamp (nullab

#### 4.2 Transformación
##### 4.2.1 Transaformacion T1
Cruzar con las tablas transaccionales de Clientes, CategoriaClientes, GruposCompras  y la tabla dimensional de Ciudades

In [22]:
clientes_df = clientes_df.join(gruposCompra_df, how = 'left', on = 'ID_GrupoCompra')
clientes_df = clientes_df.join(categoriasCliente_df, how = 'left', on = 'ID_Categoria') 
clientes_df = clientes_df.join(ciudades_dwh_df, how = 'left', on = 'ID_Ciudad_T')
clientes_df = clientes_df.selectExpr('ID_Cliente_T','Nombre','NombreCategoria','NombreGrupoCompra','ClienteFactura',
                                    'ID_Ciudad_T AS ID_CiudadEntrega_DWH','LimiteCredito','FechaAperturaCuenta','DiasPago')

clientes_df = clientes_df.fillna({'NombreCategoria':'Missing','NombreGrupoCompra':'Missing'})
clientes_df.printSchema()
print(clientes_df.count())

root
 |-- ID_Cliente_T: integer (nullable = true)
 |-- Nombre: string (nullable = true)
 |-- NombreCategoria: string (nullable = false)
 |-- NombreGrupoCompra: string (nullable = false)
 |-- ClienteFactura: integer (nullable = true)
 |-- ID_CiudadEntrega_DWH: integer (nullable = true)
 |-- LimiteCredito: decimal(10,0) (nullable = true)
 |-- FechaAperturaCuenta: timestamp (nullable = true)
 |-- DiasPago: integer (nullable = true)

663


##### 4.2.2 Transformacion T2
Adicionar Cliente con ID =0

In [23]:
# Adicionar Cliente con ID =0

schema = StructType([
    StructField("ID_Cliente_T", IntegerType(), nullable=True),
    StructField("Nombre", StringType(), nullable=True),
    StructField("NombreCategoria", StringType(), nullable=True),
    StructField("NombreGrupoCompra", StringType(), nullable=True),
    StructField("ClienteFactura", IntegerType(), nullable=True),
    StructField("ID_CiudadEntrega_DWH", IntegerType(), nullable=True),
    StructField("LimiteCredito", DecimalType(10, 0), nullable=True),
    StructField("FechaAperturaCuenta", TimestampType(), nullable=True),
    StructField("DiasPago", IntegerType(), nullable=True)
])
data = [(0, 'Missing', 'Missing','Missing',None,None, None, None, None)]
cliente_0_df = spark.createDataFrame(data, schema)
clientes_df = cliente_0_df.union(clientes_df)
print(clientes_df.head(3))
clientes_df.count()

[Row(ID_Cliente_T=0, Nombre='Missing', NombreCategoria='Missing', NombreGrupoCompra='Missing', ClienteFactura=None, ID_CiudadEntrega_DWH=None, LimiteCredito=None, FechaAperturaCuenta=None, DiasPago=None), Row(ID_Cliente_T=2, Nombre='Tailspin Toys (Sylvanite, MT)', NombreCategoria='Novelty Shop', NombreGrupoCompra='Tailspin Toys', ClienteFactura=1, ID_CiudadEntrega_DWH=33475, LimiteCredito=None, FechaAperturaCuenta=datetime.datetime(2013, 1, 1, 0, 0), DiasPago=7), Row(ID_Cliente_T=3, Nombre='Tailspin Toys (Peeples Valley, AZ)', NombreCategoria='Novelty Shop', NombreGrupoCompra='Tailspin Toys', ClienteFactura=1, ID_CiudadEntrega_DWH=26483, LimiteCredito=None, FechaAperturaCuenta=datetime.datetime(2013, 1, 1, 0, 0), DiasPago=7)]


664

#### 4.3 Carga

In [24]:
# CARGUE
guardar_db(dest_db_connection_string, clientes_df,'Estudiante_111_202315.Cliente', db_user, db_psswd)


#### 4.4 Validacion

In [25]:
#guardar_db(dest_db_connection_string, proveedores_df,'Estudiante_111_202315.Proveedor', db_user, db_psswd)
test = obtener_dataframe_de_bd(dest_db_connection_string, "Estudiante_111_202315.Cliente", db_user, db_psswd)
print(test.head(3))
test.count()

[Row(ID_Cliente_DWH=1, ID_Cliente_T=849, Nombre='Sabine Zalite', ClienteFactura=849, ID_CiudadEntrega_DWH=14377, LimiteCredito=Decimal('1500.00'), FechaAperturaCuenta=datetime.date(2013, 1, 1), DiasPago=7, NombreGrupoCompra='Missing', NombreCategoria='Supermarket'), Row(ID_Cliente_DWH=2, ID_Cliente_T=475, Nombre='Wingtip Toys (Cale, AR)', ClienteFactura=401, ID_CiudadEntrega_DWH=4900, LimiteCredito=None, FechaAperturaCuenta=datetime.date(2013, 1, 1), DiasPago=7, NombreGrupoCompra='Wingtip Toys', NombreCategoria='Novelty Shop'), Row(ID_Cliente_DWH=3, ID_Cliente_T=508, Nombre='Wingtip Toys (Branson West, MO)', ClienteFactura=401, ID_CiudadEntrega_DWH=3876, LimiteCredito=None, FechaAperturaCuenta=datetime.date(2013, 1, 1), DiasPago=7, NombreGrupoCompra='Wingtip Toys', NombreCategoria='Novelty Shop')]


664

#### 4.5 Evidencia MySQL

Fueron almacenados 664 registros desde este notebook.


![Numero Registros  Clientes MySql](https://raw.githubusercontent.com/EduardoToledoMIAD/ETL/main/Semana_4/images/Cliente_MySQL.png)

### 5. Dimension: PRODUCTOS
Su fuente de datos es la combinación entre las tablas transaccionales *Productos y Colores*

#### 5.1 Extracción

In [26]:
sql_productos = '''(SELECT DISTINCT ID_Producto as ID_Producto_T, ID_Color, NombreProducto AS Nombre, Marca, Necesita_refrigeracion AS Necesitarefrigeracion, Dias_tiempo_entrega,PrecioRecomendado AS Precio_minorista_recomendado, Impuesto, PrecioUnitario AS Precio_unitario FROM WWImportersTransactional.Producto) AS Temp_productos'''
sql_colores = '''(SELECT DISTINCT ID_Color, Color FROM WWImportersTransactional.Colores) AS Temp_colores'''

productos_df = obtener_dataframe_de_bd(source_db_connection_string, sql_productos, db_user, db_psswd)
colores_df = obtener_dataframe_de_bd(source_db_connection_string, sql_colores, db_user, db_psswd)
productos_df.printSchema()
colores_df.printSchema()


root
 |-- ID_Producto_T: integer (nullable = true)
 |-- ID_Color: integer (nullable = true)
 |-- Nombre: string (nullable = true)
 |-- Marca: string (nullable = true)
 |-- Necesitarefrigeracion: integer (nullable = true)
 |-- Dias_tiempo_entrega: integer (nullable = true)
 |-- Precio_minorista_recomendado: decimal(10,0) (nullable = true)
 |-- Impuesto: decimal(10,0) (nullable = true)
 |-- Precio_unitario: decimal(10,0) (nullable = true)

root
 |-- ID_Color: integer (nullable = true)
 |-- Color: string (nullable = true)



#### 5.2 Transformación
Cruzar Productos con Colores

In [27]:
# TRANSFORMACION

productos_df = productos_df.withColumn("cantidad_por_salida", lit(None).cast("int"))
productos_df = productos_df.join(colores_df, how = 'left', on = 'ID_Color').fillna({'Color': 'Missing'})
#productos = productos.coalesce(1).withColumn('ID_Producto_DWH', f.monotonically_increasing_id() + 1)
productos_df = productos_df.select('ID_Producto_T','Nombre','Marca','Color','Necesitarefrigeracion','Dias_tiempo_entrega', 'cantidad_por_salida','Precio_minorista_recomendado','Impuesto','Precio_unitario')
productos_df.printSchema()
print(productos_df.head(5))

productos_df.count()


root
 |-- ID_Producto_T: integer (nullable = true)
 |-- Nombre: string (nullable = true)
 |-- Marca: string (nullable = true)
 |-- Color: string (nullable = false)
 |-- Necesitarefrigeracion: integer (nullable = true)
 |-- Dias_tiempo_entrega: integer (nullable = true)
 |-- cantidad_por_salida: integer (nullable = true)
 |-- Precio_minorista_recomendado: decimal(10,0) (nullable = true)
 |-- Impuesto: decimal(10,0) (nullable = true)
 |-- Precio_unitario: decimal(10,0) (nullable = true)

[Row(ID_Producto_T=1, Nombre='USB missile launcher (Green)', Marca=None, Color='Missing', Necesitarefrigeracion=0, Dias_tiempo_entrega=14, cantidad_por_salida=None, Precio_minorista_recomendado=Decimal('37'), Impuesto=Decimal('15'), Precio_unitario=Decimal('25')), Row(ID_Producto_T=4, Nombre='USB food flash drive - sushi roll', Marca=None, Color='Missing', Necesitarefrigeracion=0, Dias_tiempo_entrega=14, cantidad_por_salida=None, Precio_minorista_recomendado=Decimal('48'), Impuesto=Decimal('15'), Precio_

227

#### 5.3 Carga

In [28]:
# CARGUE
guardar_db(dest_db_connection_string, productos_df,'Estudiante_111_202315.Producto', db_user, db_psswd)
productos_df.unpersist()

DataFrame[ID_Producto_T: int, Nombre: string, Marca: string, Color: string, Necesitarefrigeracion: int, Dias_tiempo_entrega: int, cantidad_por_salida: int, Precio_minorista_recomendado: decimal(10,0), Impuesto: decimal(10,0), Precio_unitario: decimal(10,0)]

#### 5.4. Validacion

In [29]:

test = obtener_dataframe_de_bd(dest_db_connection_string, "Estudiante_111_202315.Producto", db_user, db_psswd)
print(test.head(3))
test.count()

[Row(ID_Producto_DWH=1, ID_Producto_T=1, Nombre='USB missile launcher (Green)', Marca=None, Color='Missing', Necesitarefrigeracion=False, Dias_tiempo_entrega=14, cantidad_por_salida=None, Precio_minorista_recomendado=Decimal('37.00'), Impuesto=Decimal('15.00'), Precio_unitario=Decimal('25')), Row(ID_Producto_DWH=2, ID_Producto_T=4, Nombre='USB food flash drive - sushi roll', Marca=None, Color='Missing', Necesitarefrigeracion=False, Dias_tiempo_entrega=14, cantidad_por_salida=None, Precio_minorista_recomendado=Decimal('48.00'), Impuesto=Decimal('15.00'), Precio_unitario=Decimal('32')), Row(ID_Producto_DWH=3, ID_Producto_T=5, Nombre='USB food flash drive - hamburger', Marca=None, Color='Missing', Necesitarefrigeracion=False, Dias_tiempo_entrega=14, cantidad_por_salida=None, Precio_minorista_recomendado=Decimal('48.00'), Impuesto=Decimal('15.00'), Precio_unitario=Decimal('32'))]


227

#### 5.5 Evidencia MySQL

Fueron almacenados 227 registros desde este notebook.


![Numero Registros  Productos MySql](https://raw.githubusercontent.com/EduardoToledoMIAD/ETL/main/Semana_4/images/Producto_MySQL.png)

### 6. Dimension FECHA
Su fuente de datos son las fechas encontradas en el campo de FechaTransaccion de la tabla transaccional de Movimientos

#### 6.1 Extracción

In [33]:

sql_movimientos = '''(SELECT  FechaTransaccion FROM WWImportersTransactional.movimientos) AS Temp_movimientos'''
movimientos_df = obtener_dataframe_de_bd(db_connection_string, sql_movimientos, db_user, db_psswd)
movimientos_df.printSchema()
movimientos_df.count()


root
 |-- FechaTransaccion: string (nullable = true)



267300

#### 6.2 Transformación

T1. Estandarizar la fecha. Se evidencia dos tipos de formato de fecha  en memoria de Movimientos
- YYYY-MM-DD HH:mm:SS.000000
- MMM dd, YYYY



##### 6.2.1 Transformación T1

T1. Estandarizar la fecha. Se evidencia dos tipos de formato de fecha en memoria de Movimientos
- YYYY-MM-DD HH:mm:SS.000000
- MMM dd, YYYY



In [34]:
# TRANSFORMACION 
#T1.

regex = "\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}"
cumpleformato_df = movimientos_df.filter(movimientos_df["FechaTransaccion"].rlike(regex))
print(cumpleformato_df.count())

def estandarizar_fecha(fecha):
    regex= "\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}"
    match = re.match(regex, fecha)
    if match:
        nueva_fecha= fecha[0:19]
        return datetime.strptime(nueva_fecha, "%Y-%m-%d %H:%M:%S")
    else:
        regex = "^(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s[0-3]?[0-9],[0-9]{4}$"
        match = re.match(regex, fecha)
        if match:
            return datetime.strptime(fecha, "%b %d,%Y")
        else:
            return None;

func = udf(estandarizar_fecha, DateType())
movimientos_df=movimientos_df.withColumn('fecha_estandar',func(col('FechaTransaccion')))
movimientos_df.printSchema()
movimientos_df.count()


203046
root
 |-- FechaTransaccion: string (nullable = true)
 |-- fecha_estandar: date (nullable = true)



267300

##### 6.2.2 Transformación T2
- Una vez estandarizado las fechas en Movimientos, obtener las fechas no duplicadas y renombrar como Fecha
- Esas fechas distintas se les calcula Anyo, Mes , Dia y Numero_Semana_ISO
- A estas fechas no duplicadas  estandarizadas  con formato YYYYMMDS se convierten a integer para ser el Key de la tabla de dimension FECHA y se renombra como ID_Fecha

In [35]:
# T2.
fechas_df =movimientos_df.selectExpr('fecha_estandar as Fecha').distinct()
fechas_df = fechas_df.selectExpr('Fecha', "day(Fecha) as Dia", "month(Fecha) as Mes","year(Fecha) as Anyo","weekofyear(Fecha) as Numero_Semana_ISO" )
fechas_df = fechas_df.withColumn("ID_Fecha", date_format(col("Fecha"), "yyyyMMdd"))
fechas_df = fechas_df.withColumn("ID_Fecha", col("ID_Fecha").cast('int'))
fechas_df = fechas_df.select(col('ID_Fecha'),col('Fecha'), col('Dia'),col('Mes'), col('Anyo'), col('Numero_Semana_ISO'))
fechas_df.printSchema()
fechas_df.head(2)
fechas_df.count()
#

root
 |-- ID_Fecha: integer (nullable = true)
 |-- Fecha: date (nullable = true)
 |-- Dia: integer (nullable = true)
 |-- Mes: integer (nullable = true)
 |-- Anyo: integer (nullable = true)
 |-- Numero_Semana_ISO: integer (nullable = true)



1070

#### 6.3 Carga

In [36]:
# CARGUE

guardar_db(dest_db_connection_string, fechas_df,'Estudiante_111_202315.Fecha', db_user, db_psswd)
fechas_df.unpersist()

DataFrame[ID_Fecha: int, Fecha: date, Dia: int, Mes: int, Anyo: int, Numero_Semana_ISO: int]

#### 6.4 Validacion

In [37]:
test = obtener_dataframe_de_bd(dest_db_connection_string, "Estudiante_111_202315.Fecha", db_user, db_psswd)
test.show(5)
print(f'Numero de registros {test.count()}');
test.agg({'Fecha': 'min'}).show()


+--------+----------+---+---+----+-----------------+
|ID_Fecha|     Fecha|Dia|Mes|Anyo|Numero_Semana_ISO|
+--------+----------+---+---+----+-----------------+
|20130101|2013-01-01|  1|  1|2013|                1|
|20130102|2013-01-02|  2|  1|2013|                1|
|20130103|2013-01-03|  3|  1|2013|                1|
|20130104|2013-01-04|  4|  1|2013|                1|
|20130105|2013-01-05|  5|  1|2013|                1|
+--------+----------+---+---+----+-----------------+
only showing top 5 rows

Numero de registros 1070
+----------+
|min(Fecha)|
+----------+
|2013-01-01|
+----------+



##### 6.4.1 Validacion Regla Negocio:
*Se valida esta regla de negocio tambien: La falta de datos antes del 2014 es un error de extracción de datos. Los nuevos datos incluyen este año*. 

La fecha minima es  Jan 1 2013

#### 6.5 Evidencia MySQL

Fueron almacenados 1070 registros desde este notebook.


![Numero Registros  Fechas MySql](https://raw.githubusercontent.com/EduardoToledoMIAD/ETL/main/Semana_4/images/Fecha_MySQL.png)

### 7. Hechos: HECHO_MOVIMIENTO
Su fuente de datos es la combinación entre las tablas dimensionales  Fecha, Productos, Proveedores, Clientes, TiposTransaccion y la tabla transaccional Movimientos

#### 7.1 Extracción

In [47]:

sql_movimientos = '''(SELECT DISTINCT ProductoID, TipoTransaccionID,ClienteID,ProveedorID, FechaTransaccion,Cantidad FROM WWImportersTransactional.movimientos) AS Temp_movimientos'''
sql_productos = '''(SELECT DISTINCT ID_Producto_DWH,ID_Producto_T FROM Estudiante_111_202315.Producto) AS Temp_producto'''
sql_proveedores =  '''(SELECT DISTINCT  ID_Proveedor_DWH, ID_Proveedor_T FROM Estudiante_111_202315.Proveedor) AS Temp_proveedor'''
sql_clientes =  '''(SELECT DISTINCT ID_Cliente_DWH, ID_Cliente_T FROM Estudiante_111_202315.Cliente) AS Temp_cliente'''
sql_tipostrans= '''(SELECT DISTINCT ID_Tipo_transaccion_DWH, ID_Tipo_transaccion_T FROM Estudiante_111_202315.TipoTransaccion) AS Temp_tipotran'''

movimientos_df = obtener_dataframe_de_bd(source_db_connection_string, sql_movimientos, db_user, db_psswd)
movimientos_df.printSchema()
productos_dwh_df= obtener_dataframe_de_bd(dest_db_connection_string, sql_productos, db_user, db_psswd)
productos_dwh_df.printSchema()
proveedores_dwh_df= obtener_dataframe_de_bd(dest_db_connection_string, sql_proveedores, db_user, db_psswd)
proveedores_dwh_df.printSchema()
clientes_dwh_df= obtener_dataframe_de_bd(dest_db_connection_string, sql_clientes, db_user, db_psswd)
clientes_dwh_df.printSchema()
tipotran_dwh_df= obtener_dataframe_de_bd(dest_db_connection_string, sql_tipostrans, db_user, db_psswd)
tipotran_dwh_df.printSchema()



root
 |-- ProductoID: integer (nullable = true)
 |-- TipoTransaccionID: integer (nullable = true)
 |-- ClienteID: double (nullable = true)
 |-- ProveedorID: string (nullable = true)
 |-- FechaTransaccion: string (nullable = true)
 |-- Cantidad: double (nullable = true)

root
 |-- ID_Producto_DWH: long (nullable = true)
 |-- ID_Producto_T: integer (nullable = true)

root
 |-- ID_Proveedor_DWH: long (nullable = true)
 |-- ID_Proveedor_T: integer (nullable = true)

root
 |-- ID_Cliente_DWH: long (nullable = true)
 |-- ID_Cliente_T: integer (nullable = true)

root
 |-- ID_Tipo_transaccion_DWH: long (nullable = true)
 |-- ID_Tipo_transaccion_T: integer (nullable = true)



#### 7.2 Transformaciones

##### 7.2.1 Reglas de Negocio: Validar que Movimientos no tenga duplicados

In [48]:

# VErficar no duplicados
grupo_df = movimientos_df.groupBy(movimientos_df.columns)
cuentagrupos_df = grupo_df.count()
#Obtener si hay duplicados
cuentagrupos_df.filter(cuentagrupos_df["count"] > 1).show()


+----------+-----------------+---------+-----------+----------------+--------+-----+
|ProductoID|TipoTransaccionID|ClienteID|ProveedorID|FechaTransaccion|Cantidad|count|
+----------+-----------------+---------+-----------+----------------+--------+-----+
+----------+-----------------+---------+-----------+----------------+--------+-----+



##### 7.2.2 Transformacion: 
- Todos los provedores vacios en movimientos asignarle 0 a ID_Proveedor_T
- Todos los clientes nulos en movimientos asignarle 0 a ID_Cliente_T
- Estandarizar fecha  en la columna Fecha
- Renombrar ProductID como ID_Producto_T
- Renombrar TipoTransaccionID como ID_Tipo_transaccion_T
- Renombrar ClienteID como ID_Cliente_T
- Renombrar as ProveedorID as ID_Proveedor_T

In [49]:
movimientos_df = movimientos_df.withColumn("ProveedorID", when(col("ProveedorID") == "", 0).otherwise(col('ProveedorID').cast("int")))
movimientos_df = movimientos_df.withColumn("ClienteID", when(col("ClienteID").isNull(), 0).otherwise(col('ClienteID').cast("int")))
movimientos_df=movimientos_df.withColumn('Fecha',func(col('FechaTransaccion')))
movimientos_df= movimientos_df.selectExpr('ProductoID as ID_Producto_T','TipoTransaccionID as ID_Tipo_transaccion_T', 'ClienteID as ID_Cliente_T','ProveedorID as ID_Proveedor_T', 'Fecha',"day(Fecha) as Dia", "month(Fecha) as Mes","year(Fecha) as Anyo","Cantidad" )
movimientos_df.printSchema()


root
 |-- ID_Producto_T: integer (nullable = true)
 |-- ID_Tipo_transaccion_T: integer (nullable = true)
 |-- ID_Cliente_T: integer (nullable = true)
 |-- ID_Proveedor_T: integer (nullable = true)
 |-- Fecha: date (nullable = true)
 |-- Dia: integer (nullable = true)
 |-- Mes: integer (nullable = true)
 |-- Anyo: integer (nullable = true)
 |-- Cantidad: double (nullable = true)



##### 7.2.3 Transformacion: Convertir Fecha desde formato YYYYMMDD en Integer para representar la llave foranea con la Tabla dimensional Fecha. 


In [50]:
movimientos_df = movimientos_df.withColumn("ID_Fecha", date_format(col("Fecha"), "yyyyMMdd"))
movimientos_df = movimientos_df.withColumn("ID_Fecha", col("ID_Fecha").cast('int'))
movimientos_df= movimientos_df.select(col('ID_Fecha'), col('ID_Producto_T'),col('ID_Proveedor_T'),col('ID_Cliente_T'), col('ID_Tipo_transaccion_T'), col('Cantidad'))
movimientos_df.printSchema()



root
 |-- ID_Fecha: integer (nullable = true)
 |-- ID_Producto_T: integer (nullable = true)
 |-- ID_Proveedor_T: integer (nullable = true)
 |-- ID_Cliente_T: integer (nullable = true)
 |-- ID_Tipo_transaccion_T: integer (nullable = true)
 |-- Cantidad: double (nullable = true)



##### 7.2.4 Transformacion: 
- Cruzar Movimientos con Table dimensional Productos
- Cruzar Movimientos con Table dimensional Proveedores
- Cruzar Movimientos con Table dimensional Clientes
- Cruzar Movimientos con Table dimensional TipoTransaccion



In [51]:

temp_df = movimientos_df.join(productos_dwh_df, how = 'left', on = 'ID_Producto_T')
temp_df= temp_df.select(col('ID_Fecha'), col('ID_Producto_DWH'),col('ID_Proveedor_T'),col('ID_Cliente_T'), col('ID_Tipo_transaccion_T'), col('Cantidad'))
temp_df.printSchema()

temp_df = temp_df.join(proveedores_dwh_df, how = 'left', on = 'ID_Proveedor_T')
temp_df= temp_df.select(col('ID_Fecha'), col('ID_Producto_DWH'),col('ID_Proveedor_DWH'),col('ID_Cliente_T'), col('ID_Tipo_transaccion_T'), col('Cantidad'))
temp_df.printSchema()

temp_df = temp_df.join(clientes_dwh_df, how = 'left', on = 'ID_Cliente_T')
temp_df= temp_df.select(col('ID_Fecha'), col('ID_Producto_DWH'),col('ID_Proveedor_DWH'),col('ID_Cliente_DWH'), col('ID_Tipo_transaccion_T'), col('Cantidad'))
temp_df.printSchema()


temp_df = temp_df.join(tipotran_dwh_df, how = 'left', on = 'ID_Tipo_transaccion_T')
temp_df= temp_df.select(col('ID_Fecha'), col('ID_Producto_DWH'),col('ID_Proveedor_DWH'),col('ID_Cliente_DWH'), col('ID_Tipo_transaccion_DWH'), col('Cantidad'))
temp_df.printSchema()

temp_df.count()



root
 |-- ID_Fecha: integer (nullable = true)
 |-- ID_Producto_DWH: long (nullable = true)
 |-- ID_Proveedor_T: integer (nullable = true)
 |-- ID_Cliente_T: integer (nullable = true)
 |-- ID_Tipo_transaccion_T: integer (nullable = true)
 |-- Cantidad: double (nullable = true)

root
 |-- ID_Fecha: integer (nullable = true)
 |-- ID_Producto_DWH: long (nullable = true)
 |-- ID_Proveedor_DWH: long (nullable = true)
 |-- ID_Cliente_T: integer (nullable = true)
 |-- ID_Tipo_transaccion_T: integer (nullable = true)
 |-- Cantidad: double (nullable = true)

root
 |-- ID_Fecha: integer (nullable = true)
 |-- ID_Producto_DWH: long (nullable = true)
 |-- ID_Proveedor_DWH: long (nullable = true)
 |-- ID_Cliente_DWH: long (nullable = true)
 |-- ID_Tipo_transaccion_T: integer (nullable = true)
 |-- Cantidad: double (nullable = true)

root
 |-- ID_Fecha: integer (nullable = true)
 |-- ID_Producto_DWH: long (nullable = true)
 |-- ID_Proveedor_DWH: long (nullable = true)
 |-- ID_Cliente_DWH: long (nulla

236656

In [52]:
#Verificar que todos los proveedores de movimientos esten en Proveedores [CONSISTENCIA DE CONTENIDO]
ids_proveedores_mov = set([x.ID_Proveedor_T for x in movimientos_df.select('ID_Proveedor_T').collect()])
ids_proveedores = set([x.ID_Proveedor_T for x in proveedores_dwh_df.select('ID_Proveedor_T').collect()])


ids_proveedores_mov-ids_proveedores



set()

In [53]:
#Verificar que todos los clientes  de movimientos esten en Clientes [CONSISTENCIA DE CONTENIDO]
ids_clientes_mov = set([x.ID_Cliente_T for x in movimientos_df.select('ID_Cliente_T').collect()])
ids_clientes = set([x.ID_Cliente_T for x in clientes_dwh_df.select('ID_Cliente_T').collect()])


ids_clientes_mov-ids_clientes

set()

#### 7.3 Carga

In [54]:
guardar_db(dest_db_connection_string, temp_df,'Estudiante_111_202315.Hecho_Movimiento', db_user, db_psswd)#
temp_df.unpersist()



DataFrame[ID_Fecha: int, ID_Producto_DWH: bigint, ID_Proveedor_DWH: bigint, ID_Cliente_DWH: bigint, ID_Tipo_transaccion_DWH: bigint, Cantidad: double]

#### 7.4 Validacion

In [56]:
test = obtener_dataframe_de_bd(dest_db_connection_string, "Estudiante_111_202315.Hecho_Movimiento", db_user, db_psswd)
test.show(5)
print(f'Numero de registros {test.count()}');

+--------+---------------+----------------+--------------+-----------------------+--------+
|ID_Fecha|ID_Producto_DWH|ID_Proveedor_DWH|ID_Cliente_DWH|ID_Tipo_transaccion_DWH|Cantidad|
+--------+---------------+----------------+--------------+-----------------------+--------+
|20140917|            181|              14|           610|                      9|     -60|
|20141107|            181|              14|           280|                      9|     -60|
|20150714|            181|              14|           239|                      9|     -60|
|20151201|            181|              14|           423|                      9|     -60|
|20140422|            181|              14|           608|                      9|     -60|
+--------+---------------+----------------+--------------+-----------------------+--------+
only showing top 5 rows

Numero de registros 236656


#### 7.5 Evidencia MySQL

Fueron almacenados 236656 registros desde este notebook.


![Numero Registros  Fechas MySql](https://raw.githubusercontent.com/EduardoToledoMIAD/ETL/main/Semana_4/images/Hecho_Mov_Conteo1_MySql.png)


- Tambien se valida la regla:La falta de datos antes del 2014 es un error de extracción de datos. Los nuevos datos incluyen este año.

![Numero Fecha Minima MySql](https://raw.githubusercontent.com/EduardoToledoMIAD/ETL/main/Semana_4/images/Hecho_Mov_Fecha_Min_MySql.png)

In [57]:
spark.stop()